### Tutorial on Space-time FEM with FEniCSx
Dominik Kern ORCID [0000-0002-1958-2982](https://orcid.org/0000-0002-1958-2982) 

**solving the non-dimensional heat equation in a 1D bar
using finite elements in space and time-stepping by  the mid-point rule (Crank-Nicolson)** 

#### parameters

#### discretization

##### solution

#### post-processing